# 12-02 依赖注入与架构模式

实现松耦合的代码架构。

## 1. 依赖注入基础

In [ ]:
// 不使用 DI - 紧耦合
class UserService {
  private db = new Database();  // 直接创建依赖
}

// 使用 DI - 松耦合
interface Database {
  query(sql: string): any[];
}

class UserServiceDI {
  constructor(private db: Database) {}
  
  getUsers() {
    return this.db.query('SELECT * FROM users');
  }
}

// 可以注入不同的实现
const service1 = new UserServiceDI(new PostgresDB());
const service2 = new UserServiceDI(new MockDB());  // 测试用

## 2. 容器模式

In [ ]:
class Container {
  private services = new Map();
  
  register<T>(token: string, factory: () => T): void {
    this.services.set(token, factory);
  }
  
  resolve<T>(token: string): T {
    const factory = this.services.get(token);
    if (!factory) throw new Error(`Service ${token} not found`);
    return factory();
  }
}

// 使用
const container = new Container();
container.register('db', () => new Database());
container.register('userService', () => new UserServiceDI(container.resolve('db')));

const userService = container.resolve<UserServiceDI>('userService');

## 3. OpenClaw 的 DI 实现

In [ ]:
// OpenClaw 风格的依赖创建
interface Deps {
  db: Database;
  redis: RedisClient;
  llm: LLMProvider;
  messageRouter: MessageRouter;
}

export function createDefaultDeps(): Deps {
  const db = createDatabase();
  const redis = createRedisClient();
  const llm = createOpenAIProvider();
  const messageRouter = new MessageRouter({ db, redis, llm });
  
  return { db, redis, llm, messageRouter };
}

// 测试时注入 Mock
export function createTestDeps(): Deps {
  return {
    db: new MockDatabase(),
    redis: new MockRedis(),
    llm: new MockLLM(),
    messageRouter: new MessageRouter(mockDeps)
  };
}

## 练习

1. 实现一个简单的 DI 容器
2. 重构现有代码使用 DI
3. 对比 Java Spring 的 DI